<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/11_language_modeling_with_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformerを使った言語モデル

* 参考資料: 言語モデルに関するPyTorchのチュートリアル
  * https://pytorch.org/tutorials/beginner/transformer_tutorial.html
  * モデル以外の部分は大きく変更している。

## 今回の問題設定
* トランスフォーマを使って自分で言語モデルをtrainingする。
* 言語モデルにテキストを生成させてみる。

**ランタイムのタイプをGPUにしておく。**

## インストール

* 今回、データセットはHugging Faceのdatasetsライブラリを使って取得する。

In [ ]:
!pip install datasets

## 準備

In [ ]:
import os
import time
import math
from tqdm.notebook import tqdm
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from datasets import load_dataset

torch.manual_seed(123)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

## データセット
* Hugging Face Hubからwikitext-103-v1を取得する。
  * https://huggingface.co/datasets/wikitext
  * 約1分待つ。

In [ ]:
dataset = load_dataset("wikitext", "wikitext-103-v1")

In [ ]:
dataset

In [ ]:
dataset['train']['text'][3]

In [ ]:
min_len = 1000000
max_len = 0
for text in dataset["train"]["text"]:
  max_len = max(len(text), max_len)
  min_len = min(len(text), min_len)
print(min_len, max_len)

* すべてのテキストを繋いで、それを固定長のテキストに分けることにする。

## トークナイザの学習
* BPEアルゴリズムを使ってみる。
  * https://huggingface.co/docs/tokenizers/main/en/quicktour
* データの渡し方については、下記のリンク先を参照。
  * https://huggingface.co/learn/nlp-course/chapter6/2


* BPEトークナイザのインスタンスを作成

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

* 今回は必要でない特殊トークンもあるが、参考までに。

In [ ]:
from tokenizers.trainers import BpeTrainer
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

* プレトークナイザの設定

In [ ]:
from tokenizers.pre_tokenizers import Whitespace
tokenizer.pre_tokenizer = Whitespace()

* 1000個ずつテキストを取得するヘルパ関数
  * 処理を速めるため。

In [ ]:
def get_training_corpus():
  for start_idx in range(0, len(dataset["train"]), 1000):
    samples = dataset["train"][start_idx:start_idx+1000]
    yield samples["text"]

* 訓練データを使って、トークナイザに語彙を学習させる。
  * 90秒ぐらい待つ。

In [ ]:
tokenizer.train_from_iterator(get_training_corpus(), trainer)

* トークナイザをJSON形式のファイルとして保存。

In [ ]:
tokenizer.save("tokenizer-wiki.json")

* 語彙サイズのチェック
  * デフォルトの設定をそのまま使っただけ。

In [ ]:
tokenizer.get_vocab_size()

* トークン化を試してみる。

In [ ]:
output = tokenizer.encode(dataset["train"][3]["text"])

In [ ]:
print(output.tokens)

In [ ]:
print(output.ids)

## モデル




* PyTorchの`nn.TransformerEncoder`を使う
  * https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoderLayer.html
  * デフォルトの設定で``batch_first=False``になっていることに注意。
  * デフォルトの設定がこうなっているのは、少し前までのトークン列の扱い方の名残。

### モデルの定義

In [ ]:
class TransformerModel(nn.Module):

  def __init__(self, ntoken, d_model, nhead, d_hid, nlayers, dropout=0.5):
    super().__init__()
    # 入力されるベクトルの次元（今回はtoken embeddingの次元）
    self.d_model = d_model
    # 位置エンコーディング
    self.pos_encoder = PositionalEncoding(d_model, dropout)
    # 多層のエンコーダを作成
    encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout,
        batch_first=True, # batch_firstはTrueに変えておく。
        )
    self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
    # 入力の埋め込み層
    self.encoder = nn.Embedding(ntoken, d_model)
    # 単語ロジットを出力する全結合層（ntokenは語彙サイズ）
    self.decoder = nn.Linear(d_model, ntoken)
    # 今回は、自前の初期化を使ってみる
    self.init_weights()

  def init_weights(self):
    initrange = 0.1
    self.encoder.weight.data.uniform_(- initrange, initrange)
    self.decoder.bias.data.zero_()
    self.decoder.weight.data.uniform_(- initrange, initrange)

  def forward(self, src):
    src = self.encoder(src) * math.sqrt(self.d_model)
    src = self.pos_encoder(src)
    """Generate a square causal mask for the sequence. The masked positions are filled with float('-inf').
    Unmasked positions are filled with float(0.0).
    """
    seq_len = src.shape[1]
    src_mask = nn.Transformer.generate_square_subsequent_mask(seq_len).to(device)
    output = self.transformer_encoder(src, src_mask)
    output = self.decoder(output)
    return output

## 位置エンコーディング
* シーケンス内でのトークンの絶対的な位置をベクトルで表現する。
  * 参考資料: https://cvml-expertguide.net/terms/dl/seq2seq-translation/transformer/positional-encoding/
* 最近のLLMでは、絶対的な位置の情報は使わないことが多い。
  * 参考資料: https://pub.towardsai.net/the-quest-to-have-endless-conversations-with-llama-and-chatgpt-%EF%B8%8F-81360b9b34b2

In [ ]:
class PositionalEncoding(nn.Module):

  def __init__(self, d_model, dropout=0.1, max_len=5000):
    super().__init__()
    self.dropout = nn.Dropout(p=dropout)
    position = torch.arange(max_len).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
    pe = torch.zeros(max_len, 1, d_model)
    pe[:,0,0::2] = torch.sin(position * div_term)
    pe[:,0,1::2] = torch.cos(position * div_term)
    # `register_buffer()`を使ってpeをこのモジュールのパラメータの一部にする。
    self.register_buffer('pe', pe)

  def forward(self, x):
    # テンソルxの形は[seq_len, batch_size, embedding_dim]
    x = x + self.pe[:x.size(0)]
    return self.dropout(x)

## データセットのミニバッチ化

### トークン化をおこなうヘルパ関数

* テキストをトークン化しながら、一つの長いトークンIDの列を作る。
  * 訓練データのテキストは最初の10万件だけを使う。
  * これは単に説明の時間を短縮するため。（本当は訓練データ全体を使う。）
* トークンIDの列は、PyTorchのテンソルへ変換しておく。

In [ ]:
def data_process(data_slice, seq_len):
  token_ids = []
  # training setは大きいので、100000テキストだけ使うことにする。
  for text in tqdm(data_slice["text"][:100000]):
    token_ids += tokenizer.encode(text).ids
  truncated_length = (len(token_ids) // seq_len) * seq_len
  token_ids = torch.tensor(token_ids[:truncated_length])
  # `t()`は転置をとる操作
  return token_ids.reshape(-1, seq_len)

### データセットのトークン化

In [ ]:
# シーケンスの最大長
max_seq_len = 128

sequences = {}
for key in dataset:
  # 1を足しているのは、入力とターゲットのペアを作るとき、
  # それぞれ、最初の1トークンと、最後の1トークンを、削除するため。
  sequences[key] = data_process(dataset[key], max_seq_len + 1)

In [ ]:
sequences["train"][0]

In [ ]:
tokenizer.decode(list(sequences["train"][0]))

In [ ]:
batch_size = 16

loader = {}
for key in sequences:
  loader[key] = DataLoader(
      sequences[key],
      batch_size=batch_size,
      shuffle=(key == "train"),
      )

In [ ]:
batch = next(iter(loader["train"]))
print(batch)

In [ ]:
for token_ids in batch:
  print(tokenizer.decode(list(token_ids)))

* 入力は、後でこうやって作る。

In [ ]:
batch[:,:-1]

* ターゲットは、後でこうやって作る。

In [ ]:
batch[:,1:]

## モデルの作成




In [ ]:
vocab_size = tokenizer.get_vocab_size()  # 語彙サイズ
embed_size = 256  # トークンembeddingの次元
hidden_dim = 256  # nn.TransformerEncoderの隠れ状態のサイズ
n_layers = 2  # nn.TransformerEncoderLayerの層の数
n_head = 2  # nn.MultiheadAttentionのヘッドの数
dropout = 0.1  # dropoutの確率
model = TransformerModel(vocab_size, embed_size, n_head, hidden_dim, n_layers, dropout).to(device)

## モデルの訓練



### 損失関数と最適化アルゴリズム

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

### 訓練のためのヘルパ関数

In [ ]:
def train(model):
  model.train()  # 訓練モード
  total_loss = 0.0
  log_interval = 200
  start_time = time.time()

  num_batches = len(loader["train"])
  for i, batch in enumerate(loader["train"]):
    batch = batch.to(device)
    input, target = batch[:,:-1], batch[:,1:]
    output = model(input)
    loss = criterion(output.reshape(-1, vocab_size), target.reshape(-1))
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optimizer.step()

    total_loss += loss.item()
    if i % log_interval == 0 and i > 0:
      lr = scheduler.get_last_lr()[0]
      ms_per_batch = (time.time() - start_time) * 1000 / log_interval
      cur_loss = total_loss / log_interval
      ppl = math.exp(cur_loss)
      print(f'| epoch {epoch:3d} | {i:5d}/{num_batches:5d} batches | '
            f'lr {lr:.3e} | ms/batch {ms_per_batch:5.2f} | '
            f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
      total_loss = 0
      start_time = time.time()

### 評価のためのヘルパ関数

In [ ]:
def evaluate(model, eval_loader):
  model.eval()  # 評価モード
  total_loss = 0.0
  total_seq_len = 0
  with torch.no_grad():
    for batch in eval_loader:
      batch = batch.to(device)
      seq_len = batch.shape[0] - 1
      input, target = batch[:,:-1], batch[:,1:]
      output = model(input)
      loss = criterion(output.reshape(-1, vocab_size), target.reshape(-1))
      total_loss += seq_len * loss.item()
      total_seq_len += seq_len
  return total_loss / total_seq_len

### 学習の実行


* モデルを保存するパスの設定

In [ ]:
working_directory = os.getcwd() # ここを自分のGoogle Driveのフォルダに変更
best_model_params_path = os.path.join(working_directory, "best_model_params.pt")
print(f"save path: {best_model_params_path}")

* trainingのループを動かす。

In [ ]:
best_val_loss = float('inf')
epochs = 3
best_model = model

for epoch in range(1, epochs + 1):
  epoch_start_time = time.time()
  train(model)
  val_loss = evaluate(model, loader["validation"])
  val_ppl = math.exp(val_loss)
  elapsed = time.time() - epoch_start_time
  print('-' * 89)
  print(
      f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
      f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}'
      )
  print('-' * 89)

  if val_loss < best_val_loss:
    best_val_loss = val_loss
    torch.save(model.state_dict(), best_model_params_path)

  scheduler.step()

## テストセット上での評価




In [ ]:
test_loss = evaluate(best_model, loader["test"])
test_ppl = math.exp(test_loss)
print('=' * 89)
print(
    f'| End of training | test loss {test_loss:5.2f} | '
    f'test ppl {test_ppl:8.2f}'
    )
print('=' * 89)

## テキストの生成

In [ ]:
text = "I couldn't sleep last night. Because I was"
token_ids = torch.tensor([tokenizer.encode(text).ids]).to(device)
output = model(token_ids)

In [ ]:
output.shape

In [ ]:
output[0,-1,:].argmax()

In [ ]:
tokenizer.decode([output[0,-1,:].argmax().item()])

In [ ]:
text = "I couldn't sleep last night. Because I was"
token_ids = torch.tensor([tokenizer.encode(text).ids]).to(device)
for _ in range(10):
  output = model(token_ids)
  token_ids = torch.cat([token_ids, output[0,-1,:].argmax().reshape(1,-1)], dim=1)
  print(tokenizer.decode(list(token_ids[0])))

# 課題
* 最低限、上のコードの動作確認をしよう。
* 余裕があれば、validation perplexityの値をどこまで減らせるか、チューニングしてみよう。